In [ ]:
# Get the path to the desktop directory on macOS
import os
user_directory = os.path.join(os.path.expanduser('~'), '')
os.chdir(user_directory)

In [ ]:
import matplotlib.pyplot as plt
import librosa.display
import matplotlib.pyplot
import pandas as pd
import numpy as np

def plot_spectrogram_and_signal(audio_path, df_sentences, df_words, save_directory):
    y, sr = librosa.load(audio_path, sr=None)

    for index, row in df_sentences.iterrows():
        plt.figure(figsize=(12, 8))

        # Compute the start and end sample indices for the sentence
        start_sample_sentence = librosa.time_to_samples(row['start'], sr=sr)
        end_sample_sentence = librosa.time_to_samples(row['end'], sr=sr)

        # Extract the segment of the audio corresponding to the current sentence
        y_sentence = y[start_sample_sentence:end_sample_sentence]

        # Plot spectrogram
        plt.subplot(2, 1, 1)
        hop_length = 512
        n_fft = 2048
        sr = 44100
        # Compute the spectrogram for each signal
        spec = librosa.feature.melspectrogram(y=y_sentence, sr=sr,
                                            n_fft=n_fft, hop_length=hop_length)
        spec_db = librosa.power_to_db(spec, ref=np.max)
        librosa.display.specshow(spec_db, sr=sr, x_axis='time', y_axis='mel', n_fft=n_fft, hop_length = hop_length)

        plt.title(row['text'])

        # Subplot waveform below the spectrogram
        plt.subplot(2, 1, 2)
        librosa.display.waveshow(y_sentence, sr=sr)
        plt.title('Waveform')

        # For each word, add a vertical line and text to the waveform plot
        for _, word_row in df_words.iterrows():
            word_start = word_row['start']
            word_end = word_row['end']

            # Check if the word is within the current sentence
            if word_row['start'] >= row['start'] and word_row['end'] <= row['end']:
                word_start_sample = librosa.time_to_samples(word_start - row['start'], sr=sr)
                word_end_sample = librosa.time_to_samples(word_end - row['start'], sr=sr)
                
                # Add vertical lines to both spectrogram and waveform plots
                plt.subplot(2, 1, 1)
                        # Plot vertical lines for words
                for _, word_row in df_words.iterrows():
                    if word_row['start'] >= row['start'] and word_row['end'] <= row['end']:
                        plt.axvline(librosa.samples_to_time(word_start_sample, sr=sr), color='g', linestyle='--')
                        plt.axvline(librosa.samples_to_time(word_end_sample, sr=sr), color='tab:blue', linestyle='--')

                plt.subplot(2, 1, 2)
                for _, word_row in df_words.iterrows():
                    if word_row['start'] >= row['start'] and word_row['end'] <= row['end']:
                        plt.axvline(librosa.samples_to_time(word_start_sample, sr=sr), color='g', linestyle='--')
                        plt.axvline(librosa.samples_to_time(word_end_sample, sr=sr), color='tab:blue', linestyle='--')
                        # Print the word at its respective position on the waveform
                        word_start_in_plot = word_row['start'] - row['start']
                        word_middle_in_plot = word_start_in_plot + (word_row['end'] - word_row['start']) / 2
                        plt.text(word_middle_in_plot, np.max(y_sentence) * 0.8, word_row['word'], rotation=0, ha='center')

        # Save the plot
        plt.tight_layout()
        plt.savefig(f"{save_directory}Sentence_{index+1}_spectrogram_and_signal.jpg", dpi=300)
        plt.close()

# Replace the following with your own path
audio_path = './Desktop/Folders/.wav'
df_sentences = pd.read_csv('./Desktop/Folders/.csv')
df_words = pd.read_csv('./Desktop/Folders/.csv')
save_directory = './Desktop/Folders/'

plot_spectrogram_and_signal(audio_path, df_sentences, df_words, save_directory)
